# Imports

In [1]:
from algosdk.v2client import algod
from algosdk import account

from setup.create_app import create_app
from setup.opt_in_app import opt_in_app
from setup.call_app import call_app
from setup.helper_functions import get_private_key_from_mnemonic, is_opted_in_asset, is_opted_in_app

from private_key import creator_mnemonic

# Initializing Algod Client

In [2]:
# user declared algod connection parameters. Node must have EnableDeveloperAPI set to true in its config
algod_address = "http://localhost:4001"
algod_token = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"

# initialize an algodClient
algod_client = algod.AlgodClient(algod_token, algod_address)

# Mnemonic Phrase to Private Key

In [3]:
private_key = get_private_key_from_mnemonic(creator_mnemonic)
user_address = account.address_from_private_key(private_key)

# Creating Application

In [4]:
app_id = create_app(algod_client, private_key)

Waiting for confirmation...
Transaction 6TMXORL6NANJBZL2DZBOKSAJKCEF2BHJAUYCZS7TYB3SNBR626YQ confirmed in round 18727957.
Created new app-id: 54609609
Global state: No global state at this time.


# Opting into Application

In [5]:
if is_opted_in_app(algod_client, user_address, app_id):

    result = opt_in_app(algod_client, private_key, app_id)

# Calling Application

In [ ]:
args = []

result = call_app(algod_client, private_key, app_id, [b"vote", b"choiceA"])    